# Adaptive PDE discretizations on Cartesian grids
## Volume : Non-divergence form PDEs
## Part : Time dependent optimal control
## Chapter : Boat routing, with a quadratic cost


In this notebook, we compute the optimal route for a boat under the following assumptions:
- The objective is to minimize the fuel consumption, for a prescribed the travel time, or window of travel times.
- The boat fuel consumption is locally a quadratic function of its relative velocity, w.r.t. water.
- The boat total velocity is the sum of the relative velocity w.r.t water and of a drift due to water currents.

**Discussion of the assumptions**

The assumptions chosen above lead to a specific numerical implementation, which should be compared with the other notebooks on the [static variant](../Notebooks_FMM/BoatRouting.ipynb) of the problem, and on the [generic variant](../Notebooks_FMM/BoatRouting.ipynb) involving a tabulated cost function.
- *Time dependency.* The problem considered here is time dependent. However, if time dependency is not a prerequisite for the application, then a much simpler and faster alternative numerical method can be considered, described in the [static problem notebook](../Notebooks_FMM/BoatRouting.ipynb).
- *Quadratic cost.* The boat velocity model can be complexified. In fact, the method described below can handle any convex cost, approximated in a suitable manner. However, determining this cost (which depends on the physical properties of the boat), and approximating it (fitting a convex set with an intersection of ellipsoids), may raise difficulties in itself. An fully generic approach, but more costly, is presented in the [generic cost notebook](BoatRoutingGeneric_Time.ipynb).

**Time and space complexity**

The numerical approach presented below (time dependent problem) is orders of magnitude slover than the fast marching method discussed in [the static problem notebook](../Notebooks_FMM/BoatRouting.ipynb), and uses orders of magnitude more memory. However, the time dependent method should not be immediately dismissed on this account, since there are unexploited optimization opportunities.
- *Time complexity.* The time dependent method is embarassingly parallel, may easily take advantage of GPU computing power, possibly more efficiently than what is done here. 
- *Space complexity.* Bactracking the minimal paths requires to visit the PDE solution in reverse time. Currently, we keep a complete history of the computations, which has a substantial memory cost. A more reasonable implementation may use keypoints in time, as illustrated in possibly in a recursive manner, and recompute some parts of the solution on the fly as needed.

[**Summary**](Summary.ipynb) of volume Non-Divergence form PDEs, this series of notebooks.

[**Main summary**](../Summary.ipynb) of the Adaptive Grid Discretizations 
	book of notebooks, including the other volumes.

# Table of contents
  * [1. The model and numerical scheme](#1.-The-model-and-numerical-scheme)
    * [1.1 Stability](#1.1-Stability)
    * [1.2 Optimized implementation](#1.2-Optimized-implementation)
  * [2. Constant medium in space and time](#2.-Constant-medium-in-space-and-time)
    * [2.1 Formal checks](#2.1-Formal-checks)
    * [2.2 Comparison with an exact solution](#2.2-Comparison-with-an-exact-solution)
  * [3. Varying medium in space, constant in time](#3.-Varying-medium-in-space,-constant-in-time)
    * [3.1 Calling the eikonal solver](#3.1-Calling-the-eikonal-solver)
    * [3.2 Time dependent solver](#3.2-Time-dependent-solver)
    * [3.3 Comparison of the results](#3.3-Comparison-of-the-results)
  * [4. Extracting the optimal paths](#4.-Extracting-the-optimal-paths)
    * [4.1 Numerical computation of the geodesic flow](#4.1-Numerical-computation-of-the-geodesic-flow)
    * [4.2 Backtracking the minimal paths](#4.2-Backtracking-the-minimal-paths)
  * [5. Varying medium in space and time](#5.-Varying-medium-in-space-and-time)



**Acknowledgement.** Some of the experiments presented in these notebooks are part of 
ongoing research with Ludovic Métivier and Da Chen.

Copyright Jean-Marie Mirebeau, Centre Borelli, ENS Paris-Saclay, CNRS, University Paris-Saclay

## 0. Importing the required libraries

In [1]:
import sys; sys.path.insert(0,"..") # Allow import of agd from parent directory (useless if conda package installed)
#from Miscellaneous import TocTools; print(TocTools.displayTOC('BoatRouting_Time','NonDiv'))

In [2]:
from agd import LinearParallel as lp
from agd import FiniteDifferences as fd
from agd import Selling
from agd import AutomaticDifferentiation as ad
from agd import Eikonal
from agd.Interpolation import UniformGridInterpolation as interp
from agd.ODE.backtrack import odeint_array
from agd.Plotting import savefig,quiver; #savefig.dirName = 'Images/BoatRouting'

from agd.ExportedCode.Notebooks_FMM import BoatRouting

In [3]:
import numpy as np; xp=np; allclose=np.allclose
import matplotlib.pyplot as plt

Utility functions

In [4]:
norm_infinity = ad.Optimization.norm_infinity 
def norm(x): return ad.Optimization.norm(x,axis=0) # L2 norm on first axis
from agd.ExportedCode.Notebooks_NonDiv.LinearMonotoneSchemes2D import streamplot_ij

In [5]:
# Equivalently, with Python >= 3.8, from itertools import accumulate
from agd.ExportedCode.Notebooks_NonDiv.Time1D_NonDiv import accumulate 

In [6]:
def ReloadPackages():
    from Miscellaneous.rreload import rreload
    global lp,fd,Selling,ad,odeint_array
    lp,fd,Selling,ad,odeint_array = rreload([lp,fd,Selling,ad,odeint_array],"../..")

### 0.1 Additional configuration

Uncomment the following line to use the GPU eikonal solver.

<!---
#Eikonal.dictIn.default_mode = 'gpu_transfer' # Alternatively
--->

In [7]:
#xp,plt,Eikonal,quiver,allclose = map(ad.cupy_friendly,(xp,plt,Eikonal,quiver,allclose))

Replacing numpy with cupy, set to output 32bit ints and floats by default.
Using cp.asarray(*,dtype=np.float32) as the default caster in ad.array.
Returning a copy of module matplotlib.pyplot whose functions accept cupy arrays as input.
Setting dictIn.default_mode = 'gpu' in module agd.Eikonal .
Returning a copy of function quiver which accepts cupy arrays as input.
Setting float32 compatible default values atol=rtol=1e-5 in np.allclose


Minimal path backtracking is an inherently sequential operation, so we'll do it on the CPU in any case.

In [8]:
odeint_array = ad.cupy_generic.cupy_get_args(odeint_array,iterables=(list,))

## 1. The model and numerical scheme

Our objective is to compute the total fuel consumption $C(x,t)$, needed to reach a point $x$ at time $t$, from initial conditions (discussed below). This cost obeys the Hamilton-Jacobi-Bellman PDE
$$
    \partial_t C(t,x) + \sup_v \Big( <\nabla_x C(t,x),v> - c(t,x,v)\Big) = 0,
$$
where $c(t,x,v)$ is the instantaneous fuel consumption of a boat moving at velocity $v$, around the point $x$ and at time $t$. As announced, the time independent case $c=c(x,v)$ is best handled by a different set of methods, and is discussed in [another notebook](../Notebooks_FMM/BoatRouting.ipynb).

**Discussion on the cost function.**
For simplicity, we consider a quadratic cost, in the form
$$
    c(t,x,v) = \mu(t,x) + \frac 1 2 \|v-\omega(t,x)\|_{M(t,x)}^2,
$$
where the parameters have the following interpretation:
- $\mu(t,x)$ is the fuel consumption at rest (or any other penalization of time).
- $\omega(t,x)$ is the drift velocity.
- $M(t,x)$ is a positive definite tensor describing the domain geometry.

The numerical scheme proposed below can in principle be generalized to costs of the form 
$$
    c(t,x,v) = \sup_{\lambda\in \Lambda} a_\lambda(t,x) + <b_\lambda(t,x),v> + \|v\|^2_{c_\lambda(t,x)},
$$
where $a_\lambda$, $b_\lambda$, and $c_\lambda$ are arbitrary scalar, vector, and symmetric positive matrix. Any cost (with at least quadratic growth at infinity) can be approximated in this form.

**Elimination of the supremum in the PDE.**
Using the explicit expression of the cost function, we can simplify the HJB equation obeyed by the cost function.
In the case of the quadratic cost, we obtain 
$$
    \partial_t C(t,x) + < \nabla_x C(t,x),\omega(t,x)> + \frac 1 2 \|\nabla_x C(t,x)\|_{M(x)^{-1}}^2 - \mu(t,x) = 0
$$
The following expression will be more convenient:
$$
    \partial_t C(t,x) + \frac 1 2 \|\nabla_x C(t,x) + \eta(t,x)\|_{D(t,x)}^2 = \nu(t,x),
$$
where $D(t,x) = M(t,x)^{-1}$, $\eta(t,x) = M(t,x)\omega(t,x)$ and $\nu(t,x) = \mu(t,x) + \frac 1 2 \|\omega(t,x)\|_{M(t,x)}^2.$

In the case of the general cost, one can also eliminate the sup in the PDE, by interverting a sup and an inf, under suitable assumptions.

**Numerical scheme.**
We use an explicit scheme, which is monotone under a CFL condition. The key difficulty lies in the discretization of the quadratic first order term, which we do as follows
$$
    \|\nabla C(t,x)+\eta(t,x)\|^2_{D(x)} = \sum_{1 \leq i \leq n} \lambda_i(t,x) 
    \max\Big(0, \frac{C(t,x)-C(t,x-he_i)}h + <\eta(t,x),e_i>, \frac{C(t,x)-C(t,x+he_i)} h - <\eta(t,x),e_i>\Big)^2 + O(h).
$$
We denoted by $(\lambda_i,e_i)_{i=1}^n$ a decomposition of the tensors
$$
    D(t,x) = \sum_{1 \leq i \leq n} \lambda_i(t,x) e_i e_i^T,
$$
where $\lambda_i \geq 0$ and $e_i \in Z^d$. In practice, we use Selling's decomposition, so that $n=d (d+1)/2$, and more specifically $d=2$ and $n=3$.

**Boundary conditions.**
We assign an infinite cost to all points outside the domain, a.k.a outflow boundary conditions. In other words, the boat trajectories are constrained to remain within the domain.

In [9]:
def scheme_params(model_params):
    μ,ω,M = fd.common_field(model_params, depths=(0,1,2))    
    D = lp.inverse(M)
    η = lp.dot_AV(M,ω)
    ν = μ + 0.5*lp.dot_VAV(ω,M,ω)
    
    λ,e = Selling.Decomposition(D)
    return (ν,η,D,λ,e)

def Scheme0(C,dx,params):
    """The linear part of the scheme"""
    C,_,η,_,_,e = fd.common_field((C,)+params, depths=(0,0,1,2,1,2))
    η = np.expand_dims(η,axis=1)

    dCm = -fd.DiffUpwind(C,-e,dx,padding=np.inf) + lp.dot_VV(η,e)
    dCp = -fd.DiffUpwind(C, e,dx,padding=np.inf) - lp.dot_VV(η,e)
    return dCm,dCp

def Scheme(C,dt,params,scheme0=None,dx=None,walls=False):
    """One time step of the scheme, based on the linear part."""
    dCm,dCp = Scheme0(C,dx,params) if scheme0 is None else scheme0(C)
    C,β,_,_,λ,_ = fd.common_field((C,)+params, depths=(0,0,1,2,1,2))

    dCNorm = (λ * np.maximum(0.,np.maximum(dCp,dCm))**2).sum(axis=0)
    return np.where(xp.asarray(walls),np.inf, C + dt*(β - 0.5*dCNorm))

### 1.1 Stability

**CFL stability condition**
The numerical scheme is monotone provided the time step obeys $K_0\delta t < \delta x$, where $K_0$ depends on the Lipschitz constant of the solution, w.r.t the position $x$, and on the parameters $\omega$ and $M$. The constant $K_0$ is not hard to derive theoretically, see the function below.

**Initialization.** 
From the modeling standpoint, the fuel consumption would ideally be initialized at $0$ at the seed point, and $+\infty$ anywhere else. Unfortunately, the considered numerical scheme cannot handle such initial conditions, and will immediately produce NaNs.

Instead, we propose an initialization in the following form, assuming the seed point is the origin,
$$
    C(0,x) = K\|x\|.
$$
If the constant $K$ is chosen large enough, then all optimal paths of interest end up at the origin, and the corresponding fuel consumption is identical to the one corresponding to the ideal initialization.
This approach is referred to as *exact penalization*. The adequate value of $K$ is usually found by trial and error. Theoretical bounds exist, but they are typically too pessimistic to be usable in practice.

**Stability of the Lipschitz constant**
If the initial condition is $K$-Lipschitz, and $K\geq K_1$ where $K_1$ depends on the parameters $\omega$ and $M$, then continuous solution remains $K$-Lipschitz at all times. Likewise for the numerical solution provided the time step obeys the CFL condition. 
Theoretical bounds exist for the constant $K_1$, in terms of the Lipschitz constants of $\omega$ and $M$. However, they are exponential w.r.t time and typically too pessimistic to be usable in practice. Again, a trial and error approach is preferred.

In [10]:
def CFL(K,dx,params):
    """Returns the largest time step dt for which the scheme is monotone.
    - K : Lipschitz constant of the solution w.r.t the space variable."""
    _,η,_,λ,e = params
    K_0 = (K+norm(η))*(λ*norm(e)).sum(axis=0)
    return dx/np.max(K_0)

### 1.2 Optimized implementation

We can accelerate the computations by precomputing and saving the sparse linear structure of the linear part of the scheme.

In [11]:
def MakeScheme0(C_initial,dx,params):
    """Precompute and save the linear part of the scheme"""
    C_ad = ad.Sparse.identity(constant = np.zeros_like(C_initial))
    dCm_func,dCp_func = (e.as_func() for e in Scheme0(C_ad,dx,params))
    return lambda C : (dCm_func(C.reshape(-1)),dCp_func(C.reshape(-1)))

## 2. Constant medium in space and time

We validate the numerical scheme by considering a constant medium, in space and time, for which we know the exact solution.

In [12]:
# Parameters : unit weight for time, unit horizontal drift, euclidean geometry
model_params = (1.,xp.array((1.,0.)),xp.eye(2))
params = scheme_params(model_params) 

# Coordinate system
aX,dx = xp.linspace(-1,1,retstep=True)
X = np.meshgrid(aX,aX,indexing='ij')

# Initial condition
K_Lip = 3.
C_initial = K_Lip * ad.Optimization.norm(X,axis=0)
dt = CFL(K_Lip,dx,params)

### 2.1 Formal checks

Let us perform some basic checks on the scheme. First, it must preserve positivity.

In [13]:
C1 = Scheme(C_initial,dt,params,dx=dx)
print(f"Smallest and largest value after one step : {np.min(C1)}, {np.max(C1)}")
assert np.min(C1)>=0 and np.isfinite(np.max(C1))

Smallest and largest value after one step : 0.09168653433420221, 4.235969548918751


Second, it is a monotone scheme.

In [14]:
C_initial_ad = ad.Sparse.identity(constant=C_initial)
C1_ad = Scheme(C_initial_ad,dt,params,dx=dx)
C1_ad.simplify_ad()

print(f"Smallest and largest jacobian coefficient after one step : {np.min(C1_ad.coef)}, {np.max(C1_ad.coef)}")
assert np.min(C1_ad.coef)>=0 and np.isfinite(np.max(C1_ad.coef))

Smallest and largest jacobian coefficient after one step : -0.0, 1.0


Monotony is invalidated if we choose an excessively large time step, not obeying the CFL condition. (Which is not sharp btw.)

In [15]:
C_initial_ad = ad.Sparse.identity(constant=C_initial)
C1_ad = Scheme(C_initial_ad,2*dt,params,dx=dx)
C1_ad.simplify_ad()
print(f"Smallest and largest jacobian coefficient after one step : {np.min(C1_ad.coef)}, {np.max(C1_ad.coef)}")

Smallest and largest jacobian coefficient after one step : -0.2996101379394531, 1.0


Saving the structure of the linear part of the scheme is a simple and efficient way to reduce computation time, altough it is not a game changer in this particular application. 

In [16]:
scheme0 = MakeScheme0(C_initial,dx,params)
for a,b in zip(scheme0(C_initial),Scheme0(C_initial,dx,params)): assert allclose(a,b)

In [17]:
%%timeit -n 20
Scheme(C_initial,dt,params,scheme0); pass

461 µs ± 40 µs per loop (mean ± std. dev. of 7 runs, 20 loops each)


In [18]:
%%timeit -n 20
Scheme(C_initial,dt,params,dx=dx); pass

4.39 ms ± 151 µs per loop (mean ± std. dev. of 7 runs, 20 loops each)


### 2.2 Comparison with an exact solution

A formal analysis, led in [another notebook](../Notebooks_FMM/BoatRouting.ipynb), yields the optimal route in a constant medium in space and time

In [19]:
route = BoatRouting.route_min(X,model_params)

In particular, we know the time of the longest optimal path.

In [20]:
t_max = np.max(route['time'])
t_max,dt = map(float,(t_max,dt)) # Needed for GPU version
t_range = xp.arange(0,t_max+dt,dt)

In [21]:
%%time
C_solution = ad.array(list(accumulate(
    t_range[1:],
    initial=C_initial, 
    func=lambda C_t,t: Scheme(C_t,dt,params,scheme0)
)))

Wall time: 297 ms


In [22]:
assert allclose(C_solution[0],C_initial) and allclose(C_solution[1],C1)

In [23]:
C_opt = np.min(C_solution,axis=0); 
plt.title("Minimal fuel consumption"); plt.axis('equal')
plt.contourf(*X,C_opt); plt.colorbar();

In [24]:
t_opt = t_range[np.argmin(C_solution,axis=0)]
plt.title("Arrival time for the minimal fuel consumption"); plt.axis('equal')
plt.contourf(*X,t_opt); plt.colorbar();

The numerical solution agrees with the exact solution, up to discretization error. Note that the scheme is only first order in space and time.

In [25]:
C_error = norm_infinity(C_opt-route['cost'])
t_error = norm_infinity(t_opt-route['time'])
print(f"Error on the optimal cost: {C_error}, and on the arrival time for the optimal cost: {t_error}")

Error on the optimal cost: 0.20104717891932533, and on the arrival time for the optimal cost: 0.07942051


In [26]:
assert C_error<0.25 and t_error<0.1

## 3. Varying medium in space, constant in time

We choose a constant medium in time, varying in space, so as to be able to compare our results with the static approach based on the fast marching algorithm. This test case was first introduced in the notebook on the [static variant](../Notebooks_FMM/BoatRouting.ipynb) of the problem, and is also presented in the notebook on the variant involving a [generic cost function](../Notebooks_FMM/BoatRouting.ipynb).

### 3.1 Calling the eikonal solver

We compute a reference solution based on the fast marching algorithm. The HFM library must be installed for this step to execute.

In [27]:
hfmIn = Eikonal.dictIn({
    'verbosity':0,
    'model':'Rander2', # Riemannian + drift, what is needed here
    'exportValues':1,
    'exportGeodesicFlow':1,
    'seeds':[[-1.7,0.6]], # Where to start the front propagation
})
hfmIn.SetRect([[-2,2],[-1,1]],dimx=200) # Rectangular domain
X = hfmIn.Grid() # Coordinate system
hfmIn.update({
    'tips':hfmIn.Grid(dims=(6,3)).reshape(2,-1).T, # Points from which to backtrack geodesics
    'walls':X[1]-np.abs(X[0])>=0, # Obstacles in the domain
})

In [28]:
Embed_ω = BoatRouting.Currents(*X)
Embed_M = BoatRouting.IntrinsicMetric(BoatRouting.Spherical,*X)
model_params = (1.,Embed_ω,Embed_M)
params = scheme_params(model_params)

In [29]:
arrival_time,hfmOut = BoatRouting.ArrivalTime(hfmIn,model_params)

### 3.2 Time dependent solver

In [30]:
K_Lip = 3.
C_initial = K_Lip * norm(X- fd.as_field(hfmIn['seed'],hfmIn.shape))
C_initial[hfmIn['walls']] = np.inf
dx = hfmIn['gridScale']
dt = CFL(K_Lip,dx,params)

t_max = np.nanmax(arrival_time) # Cheating a bit: the latest arrival time is already known
dx,t_max,dt = map(float,(dx,t_max,dt)) # needed for GPU version
t_range = xp.arange(0,t_max+dt,dt)

scheme0 = MakeScheme0(C_initial,dx,params)

The solution of the time dependent PDE is more compute intensive that the static one. In addition, it uses a lot of memory; see the notebook on the [generic cost variant](BoatRoutingGeneric_Time.ipynb) for an optimization in this regard.

In [31]:
%%time
C_solution = ad.array(list(accumulate(
    t_range[1:],
    initial=C_initial, 
    func=lambda C_t,t: Scheme(C_t,dt,params,scheme0,walls=hfmIn['walls'])
)))

Wall time: 1.72 s


### 3.3 Comparison of the results

The arrival times computed by both methods match fairly well in the average. The largest difference arises where close to the cut-locus. This is not surprising, since the arrival time is discontinuous along a line in this region.

In [32]:
C_opt = np.min(C_solution,axis=0)
t_opt = t_range[np.argmin(C_solution,axis=0)]

In [33]:
t_diff_avg = np.nanmean(np.abs(t_opt - arrival_time))
print(f"Average absolute difference of the arrival times {t_diff_avg}")

Average absolute difference of the arrival times 0.005664438


In [34]:
assert t_diff_avg < 0.03

In [35]:
plt.figure(figsize=[16,4])

plt.subplot(1,2,1); plt.axis('equal')
plt.title("Optimal arrival times computed by the time dependent method")
plt.contourf(*X,t_opt); plt.colorbar()

plt.subplot(1,2,2); plt.axis('equal')
plt.title("Difference of the computed optimal arrival times")
plt.contourf(*X,t_opt-arrival_time); plt.colorbar();

The computed costs agree well, except along the eastern boundary of the excluded region (observation relevant only to the CPU eikonal solver). The difficulty lies in the fact that some minimal paths go tangentially along this eastern boundary.

**Note : on CPU/GPU eikonal solvers.**
The CPU and GPU eikonal solvers are based on distinct numerical schemes. The scheme used in the GPU case is more similar in nature to the time dependent discretization considered here.

<!---
The reason, I expect, is that the strong current in the east induces very strong anisotropy in the problem. As a result, the fast-marching method uses rather wide stencils, which decrease its accuracy in this region. (We use FM-ASR, fast-marching using adaptive stencil refinement.) 
The GPU eikonal solver is based on a different numerical scheme, which may agree better here. (TODO: check)
--->

In [36]:
walls = hfmIn['walls']
C_opt[walls] = np.nan; hfmOut['values'][walls] = np.nan
C_diff_avg = np.nanmean(np.abs(C_opt - hfmOut['values']))

In [37]:
print(f"Average difference of cost {C_diff_avg}")

Average difference of cost 0.0643245836221579


In [38]:
plt.figure(figsize=[16,4])

plt.subplot(1,2,1); plt.axis('equal')
plt.title("Minimal costs computed by the time dependent method")
plt.contourf(*X,C_opt); plt.colorbar()

plt.subplot(1,2,2); plt.axis('equal')
plt.title("Difference of the computed minimal costs")
plt.contourf(*X,C_opt-hfmOut['values']); plt.colorbar();

## 4. Extracting the optimal paths

The velocity $v$ of the boat, along the optimal trajectory at a time $t$ and position $x$, is characterized by the equation
$$
    \nabla_x C(t,x) = \nabla_v c(t,x,v).
$$
Specializing to the quadratic cost, we obtain 
$$
    \nabla_x C(t,x) = M(t,x) (v - \omega(t,x)),
$$
equivalently, with $D = M^{-1}$ and $\eta = M \omega$,
$$
    v = D(t,x) (\nabla_x C(t,x) + \eta(t,x) ).
$$
We refer to $v=V(t,x)$ as the geodesic flow, although the `geodesic` term only makes sense in the static case, and extract it numerically using the above formula, and a basic first order approximation of the gradient.
Once it is known, the optimal trajectories can be backtracked by solving the ODE 
$$
    x'(t) = V(t,x(t)),
$$
in reverse time, with a prescribed terminal condition $x(t_F) = x_F$.

### 4.1 Numerical computation of the geodesic flow

The finite scheme used to implement HJB equation can be reused, in large part, to compute the flow of the trajectories.
More precisely, under suitable smoothness assumptions, one has
$$
    D(t,x) (\nabla_x C(t,x) + \eta(t,x) ) = \sum_{1 \leq i \leq n} \lambda_i(t,x) 
    \max\Big(0, \frac{C(t,x)-C(t,x-he_i)}h + <\eta(t,x),e_i>, \frac{C(t,x)-C(t,x+he_i)} h - <\eta(t,x),e_i>\Big) s_i e_i + O(h).
$$
where $s_i \in \{0,1,-1\}$ respectively depending the active value in the maximum. This formula immediately follows from the decomposition $D(t,x) = \sum_{1 \leq i \leq n} \lambda_i(t,x) e_i e_i^T$ of the tensor field $D$. (Note that, if the $i$-th maximum is zero, then the value of $s_i$ may in fact be arbitrary.)


<!---
The next function is used broadcast the arrays $D$ and $\eta$ as time and space dependent data. The following one interpolates these fields as well as the cost values, differentiates the cost, and evaluates the flow.

**Numerical scheme.**
We use an explicit scheme, which is monotone under a CFL condition. The key difficulty lies in the discretization of the quadratic first order term, which we do as follows
$$
    \|\nabla C(t,x)+\eta(t,x)\|^2_{D(x)} = \sum_{1 \leq i \leq n} \lambda_i(t,x) 
    \max\Big(0, \frac{C(t,x)-C(t,x-he_i)}h + <\eta(t,x),e_i>, \frac{C(t,x)-C(t,x+he_i)} h - <\eta(t,x),e_i>\Big)^2 + O(h).
$$
We denoted by $(\lambda_i,e_i)_{i=1}^n$ a decomposition of the tensors
$$
    D(t,x) = \sum_{1 \leq i \leq n} \lambda_i(t,x) e_i e_i^T,
$$
where $\lambda_i \geq 0$ and $e_i \in Z^d$. In practice, we use Selling's decomposition, so that $n=d (d+1)/2$, and more specifically $d=2$ and $n=3$.
--->

<!--- Earlier implementation
def as_fieldT(array,st,sx,depth):
    """
    Broadcast an array, if needed, as a time dependent field.
    st : number of time steps, sx : shape w.r.t x
    """
    s = array.shape[depth:]
    vdim = len(sx)
    if   len(s)==0:    s = (1,)*(vdim+1)
    elif len(s)==1:    s = s+(1,)*vdim
    elif len(s)==vdim: s = (1,)+s
    r = array.shape[:depth]
    return np.broadcast_to(np.reshape(array,r+s),r+(st,)+sx)

def flow(T,X,C,η,D):
    """
    Return a callable providing the velocity of the minimal path at each time and position.
    η and D expected to be time and space dependent, broadcasted otherwise.
    """
    grid = {'shape':(len(T),)+X.shape[1:], # We do not generate the full grid, to save memory
            'origin':(T[0],)+tuple(X[:,0,0]),
            'scale':(T[1]-T[0],) + tuple(X[:,1,1]-X[:,0,0]),}
    η = as_fieldT(η,len(T),X.shape[1:],depth=1)
    D = as_fieldT(D,len(T),X.shape[1:],depth=2)    
    C,η,D = (Interpolation.UniformGridInterpolation(grid,e) for e in (C,η,D))
    def call(t,x): 
        t=np.expand_dims(t,axis=0) # for concatenation
        tx = np.concatenate((t,x),axis=0)
        η_ = η(tx); D_ = D(tx)
        x_ad = ad.Dense.identity(constant=x, shape_free=(len(x),))
        tx_ad = np.concatenate((t,x_ad),axis=0) # Differentiate value w.r.t x only
        C_grad_x = C(tx_ad).gradient()
        return lp.dot_AV(D_,C_grad_x + η_)
    return call

_,η,D,_,_ = params
flow_time = flow(t_range,X,C_solution,η,D)
flow_opt_time = flow_time(t_opt,X)
--->



In [39]:
def SchemeFlow(C,params,scheme0):
    dCm,dCp = scheme0(C)
    C,_,_,_,λ,e = fd.common_field((C,)+params, depths=(0,0,1,2,1,2))
    
    s = dCm >= dCp; s = 2*s-1 
    return (λ * np.maximum(0.,np.maximum(dCp,dCm)) * s * e).sum(axis=1)

The whole, time-independent flow.

In [40]:
%%time
C_flow = ad.array([SchemeFlow(C,params,scheme0) for C in C_solution])
C_flow[:,:,walls] = 0.

Wall time: 2.66 s


The flow at the optimal arrival time.

In [41]:
iter_opt = np.argmin(C_solution,axis=0)
C_flow_opt = np.squeeze(np.take_along_axis(C_flow,np.broadcast_to(iter_opt,(1,2,*iter_opt.shape) ),axis=0),axis=0)

Comparison with the flow computed by the static solver, appropriately scaled.

In [42]:
route = BoatRouting.route_min(hfmOut['flow'],model_params)
C_flow_static = hfmOut['flow']/route['time']

The flow computed by the time dependent PDE, and by the static PDE, agree well once the latter is appropriately rescaled. The largest differences are seen at the seed, at the cut-locus (where the flow is discontinuous), and along the right side of the obstacle.

In [43]:
flow_diff_norm = norm(C_flow_opt-C_flow_static)
print(f"Average norm of difference between the flow at optimal time, and the static flow: {np.nanmean(flow_diff_norm)}")
assert np.nanmean(flow_diff_norm)<0.2

Average norm of difference between the flow at optimal time, and the static flow: 0.022371448951076546


In [44]:
plt.figure(figsize=[16,4])

plt.subplot(1,2,1); plt.axis('equal')
plt.title("Time dependent flow, at optimal time")
quiver(*X,*C_flow_opt,subsampling=(10,10))

plt.subplot(1,2,2); plt.axis('equal')
plt.title("Norm of difference between the static and time-optimal flow")
plt.contourf(*X,flow_diff_norm);

### 4.2 Backtracking the minimal paths

In order to integrate this flow, we need an ODE solver. We use here a two-lines implementation of the Euler midpoint scheme. For practical applications, a faster and more robust ODE solver is definitely needed. (The C++ implementation of the HFM library does not suffer from these flaws.)

**Note on computation time.** Currently the geodesic backtracking is quite slow. This is purely an implementation problem, since the actual amount of computation happening is quite small - at least a hundred times smaller in terms of floating point operations than for the computation of the PDE solution. We hope to fix this performance issue at some point.

<!--- Former implementation.
**Note on the tip of the obstacle.** The backtracked geodesics, from the time dependent PDE, stop when they go past the tip of the obstacle. The solution is indeed non-differentiable at this point, which makes backtracking difficult. Again, a fix is needed.

def Euler_midpoint(t,x,V,dt):
    """One step of the Euler midpoint scheme"""
    tm = t+dt/2.; xm=x+V(t,x)*dt/2. # Midpoint time and position
    return x+V(tm,xm)*dt

def odesolve(T,x,V,step):
    return np.array(list(accumulate(
        np.asarray([T[1:],np.diff(T)]).T,
        initial=x,
        func=lambda x,t_dt : step(t_dt[0],x,V,t_dt[1])
    ))).T

geodesics = []
for x in hfmIn['tips']:
    xi,_ = hfmIn.IndexFromPoint(x)
    ti = np.argmin(C_solution.__getitem__((slice(None),*xi)))
    T = t_range[:ti+1][::-1]
    geodesics.append(odesolve(T,x,flow_time,Euler_midpoint))
--->

<!--- Profiling code.
ReloadPackages()

x = hfmIn['tips'][0]
xi,_ = hfmIn.IndexFromPoint(x)
ti = np.argmin(C_solution[tuple(xi)])
T = t_range[:ti+1][::-1]

import cProfile
cProfile.run("odesolve(T,x,flow_time,Euler_midpoint)",'profile_stats')

import pstats
p = pstats.Stats('profile_stats')
p.sort_stats('time').print_stats(20)
--->



In [45]:
%%time
y0 = hfmIn['tips'].T
best_step = np.argmin(interp(X,C_solution)(y0),axis=0) # Step of the simulation minimizing cost
t_delay = len(t_range) - best_step-1 # Account for reversed time
geodesics = odeint_array(C_flow[::-1],y0,t_range[::-1],X, t_delay=t_delay, t_substeps=1)

Wall time: 1.71 s


In [46]:
plt.title("Optimal paths, for the optimal arrival time"); plt.axis('equal')
plt.contourf(*X,walls,cmap='Greys')
for geo,delay in zip(geodesics.T,t_delay): plt.plot(*geo[:,int(delay):]) # Drop the first points according to delay

The minimal paths associated to the time dependent equation, for the optimal stopping time in terms of cost, are also the minimal paths for the static eikonal equation. Numerically, we do observe such a coincidence, up to discretization error.

In [47]:
plt.title("Optimal paths, for the static problem"); plt.axis('equal')
plt.contourf(*X,walls,cmap='Greys')
for geo in hfmOut['geodesics']: plt.plot(*geo)

## 5. Varying medium in space and time

The discretization of the time dependent method, presented above, applies with almost no modification to a time dependent vector field $\omega(t,x)$, and geometry $D(t,x)$. For the time being, we refer to the notebook with on the problem variant with a [generic cost function](BoatRoutingGeneric_Time.ipynb) where such an example is presented.